In [ ]:
import os
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    matthews_corrcoef, f1_score, precision_score, recall_score,
    accuracy_score, roc_auc_score, average_precision_score
)

from pyod.models.ae1svm import AE1SVM
from pyod.models.devnet import DevNet
from pyod.models.lunar import LUNAR
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.deep_svdd import DeepSVDD
from pyod.models.hbos import HBOS
from pyod.models.pca import PCA
from pyod.models.sod import SOD
from pyod.models.cof import COF
from pyod.models.loda import LODA
from pyod.models.alad import ALAD
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.so_gaal import SO_GAAL  # đúng: "sogaal" (không phải "so_gaal")
from pyod.models.mo_gaal import MO_GAAL  # đúng: "mogaal" (không phải "mo_gaal")
from pyod.models.vae import VAE
from pyod.models.suod import SUOD

import sys
import os
sys.path.append("../..")

from baseline_model.dasvdd_wrapper import DASVDD
from baseline_model.neutralad_wrapper import NeuTraLAD
from baseline_model.dif_wrapper import DIF


models_list = [
    # "CBLOF",
    # "KNN",
    # "IForest",
    # "OCSVM",
    # "LOF",
    # "DeepSVDD",
    # "HBOS",
    # "PCA",
    # "LODA",
    # "ECOD",
    # "COPOD",
    # "AutoEncoder",
    # "DevNet",
    # "AE1SVM",
    # "ALAD",
    # "SO_GAAL",
    # "MO_GAAL" # ,
    # # "VAE"
    "LUNAR",
    "DASVDD", 
    "SUOD", 
    "NeuTraLAD",     
    "DIF", 

]

output_file = f"../../Results/additionalBaselineOutlier.csv"
columns = ["Dataset", "Model", "outlier_mode", "scaler", "AUCROC", "AUCPR", "Accuracy", "MCC", "F1 Score",
           "Precision", "Recall", "Time Train", "Time Test"]

def preprocess_data_noise(train_data, test_data, noise_percentage=0):
    """
    Preprocess the training and testing data by separating features and labels,
    and return the preprocessed feature sets and labels with added noise.
    
    Args:
        train_data (DataFrame): Training data containing features and labels.
        test_data (DataFrame): Testing data containing features and labels.
        noise_percentage (float): The percentage of training samples to add noise to.
        
    Returns:
        X_train (ndarray): Preprocessed training features with added noise.
        y_train (ndarray): Preprocessed training labels with added noise.
        X_test (ndarray): Preprocessed testing features.
        y_test (ndarray): Preprocessed testing labels.
    """
    print("..............................Data Overview................................")
    print("Train Data Shape:", train_data.shape)
    print("Test Data Shape:", test_data.shape)
    
    # Convert to numpy arrays for easier manipulation
    X_train_total = train_data.iloc[:, :-1].to_numpy()
    y_train_total = train_data.iloc[:, -1].to_numpy()

    # Separate the samples with label 0
    X_train = X_train_total[y_train_total == 0]
    y_train = y_train_total[y_train_total == 0]

    print("Train Data Labels [0]:", np.unique(y_train))

    # Calculate how many samples to add noise to based on the provided percentage
    n_samples = X_train.shape[0]
    noise_samples_count = int(n_samples * (noise_percentage / 100))

    # Get the samples with label 1 (for generating noise)
    X_train_noise = X_train_total[y_train_total == 1]
    
    # Randomly select noise_samples_count from X_train_noise
    noisy_indices = np.random.choice(X_train_noise.shape[0], size=noise_samples_count, replace=False)
    X_train_noise = X_train_noise[noisy_indices]
    
    # Add the noisy samples to the training set
    X_train = np.vstack((X_train, X_train_noise))
    # y_train = np.concatenate((y_train, np.ones(X_train_noise.shape[0])))
    y_train = np.concatenate((y_train, np.zeros(X_train_noise.shape[0])))

    print(y_train) 
    # Prepare test data
    X_test = test_data.iloc[:, :-1].to_numpy()
    y_test = test_data.iloc[:, -1].to_numpy()

    # Print the new size of training data
    n_samples = X_train.shape[0]
    n_features = X_train.shape[1]
    print("Number of samples after adding noise:", n_samples)
    print("Number of features:", n_features)

    return X_train, y_train, X_test, y_test



def evaluate_model(y_true, y_pred, y_scores=None, y_probabilities=None):
    """
    Evaluates the model using multiple metrics and prints the results.
    This version includes AUCROC and AUCPR using predicted probabilities.

    Parameters:
    - y_true (np.ndarray): True labels of the test data.
    - y_pred (np.ndarray): Predicted labels of the test data.
    - y_scores (np.ndarray, optional): Scores used to compute AUCROC and AUCPR.
    - y_probabilities (np.ndarray, optional): Predicted probabilities for each class.

    Returns:
    - metrics (list): A list containing AUCROC, AUCPR, accuracy, MCC, F1 score, precision, and recall.
    """
    print("..............................Evaluation Metrics...............................")

    # Calculate standard metrics
    mcc = matthews_corrcoef(y_true, y_pred)  # Matthew's correlation coefficient
    f1 = f1_score(y_true, y_pred)  # F1 score
    precision = precision_score(y_true, y_pred)  # Precision
    recall = recall_score(y_true, y_pred)  # Recall
    accuracy = accuracy_score(y_true, y_pred)  # Accuracy

    # ROC and PR curve scores using predicted probabilities
    auc_roc, auc_pr = None, None
    if y_probabilities is not None:
        auc_roc = roc_auc_score(y_true, y_probabilities[:, 1])  # Probabilities for class 1 (outliers)
        auc_pr = average_precision_score(y_true, y_probabilities[:, 1])  # AUCPR for class 1

    # Display metrics
    print(f"AUCROC: {auc_roc * 100 if auc_roc else 'N/A'}")
    print(f"AUCPR: {auc_pr * 100 if auc_pr else 'N/A'}")
    print(f"Accuracy: {accuracy * 100:.2f}")
    print(f"MCC: {mcc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Return the evaluation metrics as a list
    return [auc_roc * 100 if auc_roc else None, auc_pr * 100 if auc_pr else None,
            accuracy * 100, mcc, f1, precision, recall]


def get_model(name, **kwargs):
    """
    Returns the appropriate PyOD model based on the provided name.
    
    Parameters:
    - name (str): The name of the anomaly detection model (e.g., 'CBLOF', 'KNN').
    - kwargs (dict): Additional parameters to initialize the model.

    Returns:
    - model (object): An instance of the specified anomaly detection model.
    """
    model_dict = {
        "CBLOF": CBLOF,
        "KNN": KNN,
        "IForest": IForest,
        "OCSVM": OCSVM,
        "LOF": LOF,
        "DeepSVDD": DeepSVDD,
        "HBOS": HBOS,
        "SOD": SOD,
        "COF": COF,
        "LODA": LODA,
        "PCA": PCA,
        "ECOD": ECOD,
        "COPOD": COPOD,
        "AutoEncoder": AutoEncoder,
        "DevNet": DevNet,
        "LUNAR": LUNAR,
        "AE1SVM": AE1SVM,
        "ALAD": ALAD,
        "SO_GAAL": SO_GAAL,
        "MO_GAAL": MO_GAAL,
        "VAE":VAE, 
        "DASVDD": DASVDD, 
        "SUOD": SUOD, 
        "NeuTraLAD": NeuTraLAD,
        "DIF": DIF, 
    }
    
    # Fetch the model class based on the provided model name
    model_class = model_dict.get(name)
    
    # Raise an error if the model name is not found
    if model_class is None:
        raise ValueError(f"Model {name} not found.")
    
    # Return the model initialized with the provided parameters
    return model_class(**kwargs)


# Initialize output CSV file
if not os.path.exists(output_file):
    pd.DataFrame(columns=columns).to_csv(output_file, index=False)

# Define a function to run the model training and evaluation
def run_experiment(X_train, y_train, X_test, y_test, name, noise_percentage, outlier_mode, scaler ):
    for model_name in models_list:
        try:
            print(f"\nRunning dataset {name} with model {model_name}")
            
            # train_data = pd.concat([X_train, y_train], axis=1, ignore_index=True)
            # test_data = pd.concat([X_test, y_test], axis=1, ignore_index=True)

            # # Preprocess data (assuming preprocess_data is defined elsewhere)
            # X_train, y_train, X_test, y_test = preprocess_data(train_data, test_data)
            
            if model_name == 'DeepSVDD':
                start_time = time.time()
                n_features = X_train.shape[1]
                model = DeepSVDD(n_features=n_features)
                model.fit(X_train)
            elif model_name == 'DASVDD':
                start_time = time.time()
                model = DASVDD(
                    code_size=32,
                    num_epochs=100,
                    batch_size=128,
                    lr=1e-3,
                    K=0.9,
                    T=10,
                    verbose=1
                )
                model.fit(X_train)
            elif model_name == "DIF": 
                start_time = time.time()
                model = DIF(n_ensemble=50, n_estimators=6)

                # Hoặc với custom config
                model = DIF(
                    n_ensemble=50,
                    n_estimators=6,
                    max_samples=256,
                    hidden_dim=[500, 100],
                    rep_dim=20,
                    activation='tanh',
                    batch_size=64,
                    device='cuda',
                    verbose=1
                )
                model.fit(X_train)
                
            elif model_name == "NeuTraLAD": 
                start_time = time.time()
                model = NeuTraLAD(
                    latent_dim=32,      # Kích thước latent space
                    enc_hdim=32,        # Hidden dim encoder
                    num_trans=11,       # Số transformations
                    num_epochs=200,     # Số epochs
                    batch_size=128,
                    lr=0.001,
                    device='cuda',      # hoặc 'cpu'
                    verbose=1           # 1 để hiển thị progress
                )
                # Fit model (chỉ dùng normal data)
                model.fit(X_train)
            elif model_name == "SUOD": 
                start_time = time.time()
                
                base_estimators = [
                    HBOS(n_bins=10),
                    COPOD(),
                    ECOD(),
                    IForest(n_estimators=50, max_samples=128, random_state=42),
                    PCA(n_components=10, random_state=42),
                ]
                
                model = SUOD(
                    base_estimators=base_estimators,
                    n_jobs=1,                    # BẮT BUỘC = 1 để tránh hoàn toàn bug joblib
                    rp_flag_global=True,         # vẫn tăng tốc
                    bps_flag=False,              # tắt BPS → nguyên nhân chính của lỗi
                    contamination=0.1,
                    verbose=True                 # in chi tiết để bạn thấy nó chạy
                )
                
                model.fit(X_train)  # fit all models with X
            else:
                model = get_model(model_name)
                start_time = time.time()
                if model_name == 'DevNet':
                    model.fit(X_train, y_train)
                else:
                    model.fit(X_train)
                    
            train_time = time.time() - start_time

           #  columns = ["Dataset", "Model", "outlier_mode", "scaler", "AUCROC", "AUCPR", "Accuracy", "MCC", "F1 Score",
           # "Precision", "Recall", "Time Train", "Time Test"]
            # Test the model
            start_time = time.time()
            y_pred = model.predict(X_test)
            # Check if the model supports predict_proba and calculate probabilities
            if hasattr(model, "predict_proba"):
                y_probabilities = model.predict_proba(X_test)  # Class probabilities for each instance
            else:
                y_probabilities = None  # Some models do not have predict_proba method

            test_time = time.time() - start_time

            # Evaluate the model using the appropriate probabilities
            metrics = evaluate_model(y_test, y_pred, y_scores=None, y_probabilities=y_probabilities)
            result = [name, model_name] + [outlier_mode, scaler]+ metrics + [train_time, test_time]
            result_df = pd.DataFrame([result], columns=columns)
            result_df.to_csv(output_file, mode='a', header=False, index=False)

            print(f"Results saved for {name} with model {model_name}")

        except Exception as e:
            print(f"Error with dataset {name}, model {model_name}: {e}")



if __name__ == "__main__": 
    
    dataset_prefixes =  ['data_ToNIoT.csv', 'data_CICIoT2023.csv','data_N_BaIoT.csv', 'data_BoTIoT.csv']
    scaler_names = ['QuantileTransformer','MinMaxScaler','Normalizer','RobustScaler', 'StandardScaler' ]
    
    outlier_modes = [ 'local', 'cluster', 'global']

    for scaler in scaler_names: 
    
        for prefix in dataset_prefixes:
            
            for outlier_mode in outlier_modes: 
             
                # Construct file paths for train and test datasets with 'Train_' and 'Test_' prefixes
                train_file = f'../../Datascaled/OutlierData/Train_{outlier_mode}_{scaler}_{prefix}'
                test_file = f'../../Datascaled/OutlierData/Test_{outlier_mode}_{scaler}_{prefix}'
                
                if not os.path.exists(train_file):
                    print(f"❌ File không tồn tại: {train_file}, bỏ qua.")
                    continue  # Bỏ qua vòng lặp nếu file không tồn tại

                if not os.path.exists(test_file):
                    print(f"❌ File không tồn tại: {test_file}, bỏ qua.")
                    continue  # Bỏ qua vòng lặp nếu file không tồn tại
                
                # Load the CSV files
                df_train = pd.read_csv(train_file)
                df_test = pd.read_csv(test_file)
        
                
                X_train, y_train, X_test, y_test = preprocess_data_noise(df_train, df_test)
                # Run the experiments for each dataset typ
                run_experiment(X_train, y_train, X_test, y_test, prefix, 0, outlier_mode, scaler ) 
               



..............................Data Overview................................
Train Data Shape: (3000, 29)
Test Data Shape: (7000, 29)
Train Data Labels [0]: [0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Balanced Scheduling Total Train Time: 1.8855676651000977
Split among workers default: [5] []
Split among workers default: [5] []
Parallel score prediction...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Parallel Score Prediction without Approximators Total Time: 0.8559679985046387
Split among workers default: [5] []
Parallel score prediction...
Parallel Score Prediction without Approximators Total Time: 0.06535077095031738
..............................Evaluation Metrics...............................
AUCROC: 76.89924577155108
AUCPR: 95.41309758857825
Accuracy: 26.61
MCC: 0.0715
F1 Score: 0.3139
Precision: 0.9385
Recall: 0.1885
Results saved for data_ToNIoT.csv with model SUOD

Running dataset data_ToNIoT.csv with model NeuTraLAD
Epoch 10/200, Loss: 0.464103
Epoch 20/200, Loss: 0.270389
Epoch 30/200, Loss: 0.198530
Epoch 40/200, Loss: 0.167646
Epoch 50/200, Loss: 0.154831
Epoch 60/200, Loss: 0.149075
Epoch 70/200, Loss: 0.145451
Epoch 80/200, Loss: 0.142993
Epoch 90/200, Loss: 0.141279
Epoch 100/200, Loss: 0.140064
Epoch 110/200, Loss: 0.139115
Epoch 120/200, Loss: 0.138437
Epoch 130/200, Loss: 0.137911
Epoch 140/200, Loss: 0.137470
Epoch 150/200, Loss: 0.137130
Epoch 160/200, Loss: 0

...

Parallel Score Prediction without Approximators Total Time: 0.08290672302246094
..............................Evaluation Metrics...............................
AUCROC: 99.99940292840968
AUCPR: 99.99992495763165
Accuracy: 99.87
MCC: 0.9935
F1 Score: 0.9993
Precision: 0.9986
Recall: 1.0000
Results saved for data_ToNIoT.csv with model SUOD

Running dataset data_ToNIoT.csv with model NeuTraLAD
Epoch 10/200, Loss: 0.489154
Epoch 20/200, Loss: 0.248894
Epoch 30/200, Loss: 0.166710
Epoch 40/200, Loss: 0.146906
Epoch 50/200, Loss: 0.141677
Epoch 60/200, Loss: 0.139376
Epoch 70/200, Loss: 0.138074
Epoch 80/200, Loss: 0.137319
Epoch 90/200, Loss: 0.136636
Epoch 100/200, Loss: 0.136173
Epoch 110/200, Loss: 0.135853
Epoch 120/200, Loss: 0.135585
Epoch 130/200, Loss: 0.135356
Epoch 140/200, Loss: 0.135194
Epoch 150/200, Loss: 0.135081
Epoch 160/200, Loss: 0.134956
Epoch 170/200, Loss: 0.134878
Epoch 180/200, Loss: 0.134789
Epoch 190/200, Loss: 0.134723
Epoch 200/200, Loss: 0.134676
................

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Parallel Score Prediction without Approximators Total Time: 0.09049224853515625
..............................Evaluation Metrics...............................
AUCROC: 91.20174139291943
AUCPR: 92.95001224428698
Accuracy: 86.31
MCC: 0.6942
F1 Score: 0.8986
Precision: 0.9354
Recall: 0.8646
Results saved for data_CICIoT2023.csv with model SUOD

Running dataset data_CICIoT2023.csv with model NeuTraLAD
Epoch 10/200, Loss: 0.677872
Epoch 20/200, Loss: 0.470156
Epoch 30/200, Loss: 0.333209
Epoch 40/200, Loss: 0.253190
Epoch 50/200, Loss: 0.202154
Epoch 60/200, Loss: 0.174246
Epoch 70/200, Loss: 0.156603
Epoch 80/200, Loss: 0.145660
Epoch 90/200, Loss: 0.142393
Epoch 100/200, Loss: 0.140187
Epoch 110/200, Loss: 0.138854
Epoch 120/200, Loss: 0.138005
Epoch 130/200, Loss: 0.137388
Epoch 140/200, Loss: 0.136909
Epoch 150/200, Loss: 0.136535
Epoch 160/200, Loss: 0.136229
Epoch 170/200, Loss: 0.135973
Epoch 180/200, Loss: 0.135759
Epoch 190/200, Loss: 0.135588
Epoch 200/200, Loss: 0.135443
........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Parallel Score Prediction without Approximators Total Time: 0.10135889053344727
..............................Evaluation Metrics...............................
AUCROC: 91.62278602202011
AUCPR: 93.34727238167648
Accuracy: 90.97
MCC: 0.7861
F1 Score: 0.9395
Precision: 0.8860
Recall: 1.0000
Results saved for data_CICIoT2023.csv with model SUOD

Running dataset data_CICIoT2023.csv with model NeuTraLAD
Epoch 10/200, Loss: 0.794283
Epoch 20/200, Loss: 0.572996
Epoch 30/200, Loss: 0.409124
Epoch 40/200, Loss: 0.304868
Epoch 50/200, Loss: 0.246406
Epoch 60/200, Loss: 0.212708
Epoch 70/200, Loss: 0.185134
Epoch 80/200, Loss: 0.160153
Epoch 90/200, Loss: 0.145386
Epoch 100/200, Loss: 0.141641
Epoch 110/200, Loss: 0.139118
Epoch 120/200, Loss: 0.137808
Epoch 130/200, Loss: 0.137086
Epoch 140/200, Loss: 0.136594
Epoch 150/200, Loss: 0.136240
Epoch 160/200, Loss: 0.135977
Epoch 170/200, Loss: 0.135764
Epoch 180/200, Loss: 0.135588
Epoch 190/200, Loss: 0.135442
Epoch 200/200, Loss: 0.135317
........